<a href="https://colab.research.google.com/github/chedlyba/birdclef/blob/master/birdclef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r /content/drive/MyDrive/birdclef/* .

In [3]:
!mkdir 2

In [4]:
!cp /content/drive/MyDrive/bird/2/* ./2

In [5]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 5.0 MB/s 


In [49]:
import pandas as pd 
import numpy as np
import torch
from torch import tensor
import json
from BinaryNet.model_utils import BinarizeLinearLayer, BinarizeConv2d, Adam_meta, Adam_bk, SignActivation
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn
from datetime import datetime
from torchmetrics import F1Score
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from dataset import AudioUtil


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

def load_dataset(data_path):
    with open(data_path, 'r') as fp:
        data = json.load(fp)
    # extract inputs and tagets

    X = torch.from_numpy(np.array(data['spectrogram']))
    y = data['bird']

    return X, y

def train(model, train_dl, lr=5e-03, epochs=50, optim='', meta=0.0):
 
    criterion = nn.CrossEntropyLoss()
    if optim == 'meta':
        optimizer = Adam_meta(model.parameters(), lr=lr, meta=meta, weight_decay=1e-07)
    else :
        optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-07)
    
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr= 0.005,
                                                   steps_per_epoch=int(len(train_dl)),
                                                   epochs=epochs,
                                                   anneal_strategy='linear')
    loss_vec = []
    acc_vec = []
    for epoch in range(epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0
        start = datetime.now()
        model.train()
        for i, data in enumerate(train_dl):

            inputs, labels = data[0].float().to(DEVICE), data[1].long().to(DEVICE)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            
            for p in model.parameters():  # updating the org attribute
                if hasattr(p,'org'):
                    p.data.copy_(p.org)
                    
            optimizer.step()

            for p in model.parameters():  # updating the org attribute
                if hasattr(p,'org'):
                    p.org.copy_(p.data)
            
            scheduler.step()
            running_loss += loss.item()

            _, prediction = torch.max(outputs, 1)
            print(outputs)
            print('pred \n', prediction)
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

            if i % 100 == 0 and i !=0:  
                print(f'[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
                print(f'{datetime.now()-start}')
                start = datetime.now()

        num_batches = len(train_dl)
        avg_loss = running_loss/num_batches
        acc = correct_prediction/total_prediction
        acc_vec.append(acc)   
        loss_vec.append(avg_loss)
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')
    return loss_vec, acc_vec
    
def get_data_split(data_path):
    X, y = load_dataset(data_path)
    labels= pd.unique(y).tolist()
    l = []
    for birds in labels:
        for bird in birds.split(' '):
            l.append(bird)
    labels = pd.unique(l).tolist()
        
    # create train/validation/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.3)

    X_train = torch.swapaxes(X_train, 1, -1)
    X_validation = torch.swapaxes(X_validation, 1 ,-1)
    X_test = torch.swapaxes(X_test, 1, -1)
    
    return X_train, X_validation, X_test, y_train, y_validation, y_test, labels

def normalize(tensor):
  tensor_norm = tensor - tensor.mean()
  return tensor_norm / (tensor_norm.abs().max() + 1e-7)


class SoundDS(Dataset):
    def __init__(self, x, y, mappings, is_train=True, augment=1):
        self.x = []
        self.y = []
        if is_train:
          augmented_data = []
          augmented_data_labels = []
          
          for data, label in zip(x, y):
            data = normalize(data)
            self.x.append(data)
            self.y.append(label)
            if label != 'nocall': 
              for _ in range(augment):
                  new_data = (data + torch.unsqueeze(torch.randn(data.shape[1], data.shape[2])-torch.mean(data), 0))
                  augmented_data_labels.append(label)
                  augmented_data.append(new_data)
            
          self.x += augmented_data
          self.y += augmented_data_labels

        self.mappings = mappings
        self.num_classes = len(mappings.keys())

    def __getitem__(self, idx):
        target = self.mappings[self.y[idx].split()[0]]
        return self.x[idx] , target

    def __len__(self):
        return len(self.x)

def inference(model, val_dl):
    correct_pred = 0
    total_pred = 0
    preds = np.empty(shape=(1,))
    targets = np.empty(shape=(1,))
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for data in val_dl:
            inputs, labels = data[0].float().to(DEVICE), data[1].long()
            outputs = model(inputs).to('cpu')
            
            loss = criterion(outputs, labels)
            _, prediction = torch.max(outputs,1)
            correct_pred += (prediction == labels).sum().item()
            
            total_pred += prediction.shape[0]
            
            preds = np.append(preds, prediction)
            targets = np.append(targets, labels)
    preds = preds[1:,...]
    targets = targets[1:,...]
    acc = correct_pred / total_pred
    print(f'Accuracy: {acc:.2f}, Loss: {loss:.2f}, Total items: {total_pred}')
    return preds, targets



In [40]:
a = torch.tensor([1, 2, 3])
l = []

In [43]:
a

tensor([1, 2, 3])

In [44]:
l.append(a)

In [45]:
l

[tensor(1), tensor(2), tensor(3), tensor([1, 2, 3])]

In [7]:
from torch import nn
from collections import OrderedDict

class AudioClassifierBNN(nn.Module):

    def __init__(self, init='uniform', width=0.01, n_labels=10, n_channels=1):
        super(AudioClassifierBNN, self).__init__()
        self.n_channels = n_channels
        self.hidden_layers = 4
        self.n_labels = n_labels
        self.softmax = nn.Softmax()
        self.layer_list = [
            ( ('cv1', BinarizeConv2d(self.n_channels, 64, kernel_size=(5, 5), stride = (2, 2), bias=False, padding=(2 ,2))) ),
            ( ('bn1', nn.BatchNorm2d(64)) ),
            ( ('cv2', BinarizeConv2d(64, 128, kernel_size=(3, 3), stride = (2, 2), bias=False, padding=(2 ,2))) ),
            ( ('bn2', nn.BatchNorm2d(128)) ),
            ( ('cv3', BinarizeConv2d(128,1024, kernel_size=(3, 3), stride = (2, 2), bias=False, padding=(2 ,2))) ),
            ( ('bn3', nn.BatchNorm2d(1024) ) ),
            ( ('ap', nn.AdaptiveAvgPool2d(output_size=1)) ),
            ( ('fc1', BinarizeLinearLayer(in_features=1024, out_features=32)) ),
            ( ('bn4', nn.BatchNorm1d(32)) ),
            ( ('fc2', BinarizeLinearLayer(in_features=32, out_features=n_labels)) ),
            ( ('bn5', nn.BatchNorm1d(n_labels)) ),
        ]
        
        self.layers = torch.nn.ModuleDict(OrderedDict(self.layer_list))

        for key in self.layers.keys():
            if not('bn' in key) and not('ap' in key):
                if init == "gauss":
                    torch.nn.init.normal_(self.layers[key].weight, mean=0, std=width)
                if init == "uniform":
                    torch.nn.init.uniform_(self.layers[key].weight, a= -width/2, b=width/2)
    
    def save_weight(self):
      for name, param in self.named_parameters():
        print(name, ': ', param)

    def forward(self, x):
        self.layers['cv1'].is_input = True
        x = self.layers['cv1'](x)
        x = self.layers['bn1'](x)
        x = SignActivation.apply(x)
        x = self.layers['cv2'](x)
        x = self.layers['bn2'](x)
        x = SignActivation.apply(x)
        x = self.layers['cv3'](x)
        x = self.layers['bn3'](x)
        x = self.layers['ap'](x)
        x = x.view(x.shape[0], -1)
        x = SignActivation.apply(x)
        x = self.layers['fc1'](x)
        if x.shape[0] !=1 :
            x = self.layers['bn4'](x)
        x = SignActivation.apply(x)
        x = self.layers['fc2'](x)
        if x.shape[0] !=1 :
            x = self.layers['bn5'](x)
        x = self.softmax(x)
        return x

In [8]:
class AudioClassifier(nn.Module):

    def __init__(self, init='uniform', width=0.01, n_labels=10, n_channels=1):
        super(AudioClassifier, self).__init__()
        self.n_channels = n_channels
        self.hidden_layers = 4
        self.n_labels = n_labels
        self.softmax = nn.Softmax()
        self.layer_list= [
            ( ('cv1', nn.Conv2d(self.n_channels, 64, kernel_size=(5, 5), stride = (2, 2), bias=False, padding=(2 ,2)))) ,
            ( ('bn1', nn.BatchNorm2d(64)) ),
            ( ('cv2', nn.Conv2d(64, 128, kernel_size=(3, 3), stride = (2, 2), bias=False, padding=(2 ,2))) ),
            ( ('bn2', nn.BatchNorm2d(128)) ),
            ( ('cv3', nn.Conv2d(128, 64, kernel_size=(3, 3), stride = (2, 2), bias=False, padding=(2 ,2))) ),
            ( ('bn3', nn.BatchNorm2d(64)) ),
            ( ('ap', nn.AdaptiveAvgPool2d(output_size=1)) ),
            ( ('fc1', nn.Linear(in_features=64, out_features=32)) ),
            ( ('bn4', nn.BatchNorm1d(32)) ),
            ( ('fc2', nn.Linear(in_features=32, out_features=n_labels)) ),
            ( ('bn5', nn.BatchNorm1d(n_labels)) )
        ]
        self.layers = torch.nn.ModuleDict(OrderedDict(self.layer_list))

        for key in self.layers.keys():
            if not('bn' in key) and not('ap' in key):
                if init == "gauss":
                    torch.nn.init.normal_(self.layers[key].weight, mean=0, std=width)
                if init == "uniform":
                    torch.nn.init.uniform_(self.layers[key].weight, a= -width/2, b=width/2)

    def forward(self, x):

        x = self.layers['cv1'](x)
        x = self.layers['bn1'](x)
        x = self.layers['cv2'](x)
        x = self.layers['bn2'](x)
        x = self.layers['cv3'](x)
        x = self.layers['bn3'](x)
        x = self.layers['ap'](x)
        x = x.view(x.shape[0], -1)
        x = self.layers['fc1'](x)
        if x.shape[0] !=1 :
            x = self.layers['bn4'](x)
        x = self.layers['fc2'](x)
        if x.shape[0] !=1 :
            x = self.layers['bn5'](x)
        x = self.softmax(x)
        return x

In [26]:
task = '1'

In [47]:
train_ds = SoundDS(data[task]['X_train'], data[task]['y_train'], mappings, augment=15)

In [172]:
mappings

{'annhum': 0, 'astfly': 1, 'babwar': 2, 'baleag': 3, 'balori': 4}

In [173]:
print(data['1']['X_train'][0])

tensor([[[-51.7776, -55.5423, -55.5423,  ..., -55.5423, -55.5423, -45.7676],
         [-52.4699, -55.5423, -55.5423,  ..., -55.5423, -55.5423, -45.0577],
         [-51.3451, -55.5423, -55.5423,  ..., -55.5423, -55.5423, -44.9980],
         ...,
         [-25.1322, -26.0898, -25.2648,  ..., -19.2778, -16.1323, -18.9690],
         [-27.8082, -28.0915, -28.9656,  ..., -20.4978, -22.5094, -21.4597],
         [-32.2909, -34.2286, -33.5587,  ..., -29.0875, -28.1351, -27.1220]]],
       dtype=torch.float64)


In [48]:
for i in range(len(train_ds)):
  _ =train_ds.__getitem__(i)[0]

torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])
torch.Size([1, 64, 313])


In [24]:
tasks = {
    '1': [f'data_m_{i+101}.0.json' for i in range(10)], 
    '2': [f'data_m_{i+130}.0.json' for i in range(10)]
}

PATH = './2/'

data = {}
labels = []
for task in tasks.keys():
    data[task] = {
        'X_train': [],
        'y_train': [],
        'X_test': [],
        'y_test': [],
        'X_val': [],
        'y_val': [],
        'labels': []
    }
    for path in tasks[task]:
        print(path)
        X_train, X_val, X_test, y_train, y_val, y_test, l = get_data_split(PATH + path)
        
        data[task]['X_train']+=X_train
        data[task]['y_train']+=y_train
        data[task]['X_test']+=X_test
        data[task]['y_test']+=y_test
        data[task]['X_val']+=X_val
        data[task]['y_val']+=y_val
        data[task]['labels'].append(l)
        labels.append(l)
    
    data[task]['labels'] = pd.unique( [birds.split()[0] for subset in data[task]['labels'] for birds in subset ] ).tolist()
    print(f'Task {task} labels:')
    print(data[task]['labels'])
labels = [ birds.split()[0] for subset in labels for birds in subset ]
labels = pd.unique(labels).tolist() 
print('Labels extracted : ', labels)

mappings = {}
n_labels = 0

for i, label in enumerate(labels):
    mappings[label] = i
    n_labels =1 + i


data_m_101.0.json
torch.Size([19, 1, 64, 313])
data_m_102.0.json
torch.Size([19, 1, 64, 313])
data_m_103.0.json
torch.Size([19, 1, 64, 313])
data_m_104.0.json
torch.Size([19, 1, 64, 313])
data_m_105.0.json
torch.Size([19, 1, 64, 313])
data_m_106.0.json
torch.Size([19, 1, 64, 313])
data_m_107.0.json
torch.Size([19, 1, 64, 313])
data_m_108.0.json
torch.Size([19, 1, 64, 313])
data_m_109.0.json
torch.Size([19, 1, 64, 313])
data_m_110.0.json
torch.Size([19, 1, 64, 313])
Task 1 labels:
['annhum', 'astfly']
data_m_130.0.json
torch.Size([19, 1, 64, 313])
data_m_131.0.json
torch.Size([19, 1, 64, 313])
data_m_132.0.json
torch.Size([19, 1, 64, 313])
data_m_133.0.json
torch.Size([19, 1, 64, 313])
data_m_134.0.json
torch.Size([19, 1, 64, 313])
data_m_135.0.json
torch.Size([19, 1, 64, 313])
data_m_136.0.json
torch.Size([19, 1, 64, 313])
data_m_137.0.json
torch.Size([19, 1, 64, 313])
data_m_138.0.json
torch.Size([19, 1, 64, 313])
data_m_139.0.json
torch.Size([19, 1, 64, 313])
Task 2 labels:
['babwar'

In [176]:
df_task1 = pd.DataFrame(
    {
      'X' : data['1']['X_train'],
      'y' : data['1']['y_train']
    }
)
df_task2 = pd.DataFrame(
    {
        'X' : data['2']['X_train'],
        'y' : data['2']['y_train']
    }
)

In [177]:
df_task1['y'].value_counts()

annhum    124
astfly     66
Name: y, dtype: int64

In [178]:
df_task2['y'].value_counts()

baleag    148
balori     23
babwar     19
Name: y, dtype: int64

In [85]:
df_task2.drop(df_task2[(df_task2['y']=='babwar')].index, inplace=True)
df_task2.drop(df_task2[(df_task2['y']=='banana')].index, inplace=True)

In [86]:
df_task2['y'].value_counts()

baleag    149
balori    149
Name: y, dtype: int64

In [87]:
data['2']['y_train'] = df_task2['y'].values
data['2']['X_train'] = df_task2['X'].values

In [88]:
labels = list(set(labels) - set(['babwar', 'banana']))
n_labels = 0
mappings = {}
for i, label in enumerate(labels):
    mappings[label] = i
    n_labels =1 + i

In [89]:
mappings

{'annhum': 0, 'astfly': 3, 'azaspi1': 4, 'baleag': 2, 'balori': 1}

In [ ]:
!cp model_1.h5 drive/MyDrive/birdclef
!cp model_bnn_1.h5 drive/MyDrive/birdclef
!cp model_2.h5 drive/MyDrive/birdclef
!cp model_bnn_2.h5 drive/MyDrive/birdclef


In [123]:
#del df_task1, df_task2
del modelbnn

In [50]:
loss_dict = {}
acc_dict = {}
task = '1'

print(f'Training Task {task}:')
print('Training size: ', len(data[task]['X_train']))


model = AudioClassifier(n_labels=n_labels).to(DEVICE)
modelbnn = AudioClassifierBNN(n_labels=n_labels).to(DEVICE)

train_ds = SoundDS(data[task]['X_train'], data[task]['y_train'], mappings, augment=15)
#val_ds = SoundDS(data[task]['X_val'], data[task]['y_val'], mappings, is_train=False)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
#val_dl = DataLoader(val_ds, batch_size=32, shuffle=False)

loss1, acc1 = train(model, train_dl, epochs=15, optim='')
torch.save(model.state_dict(), f'model_{task}.h5')

# del model
# #modelbnn.load_state_dict(torch.load('model_bnn_2.h5'))
# loss2, acc2= train(modelbnn, train_dl, epochs=30, optim='meta', meta=2)
# torch.save(modelbnn.state_dict(), f'model_bnn_{task}.h5')

# del modelbnn


loss_dict[task] = { 
    'model' : loss1,
#    'bnn' : loss2
    }

acc_dict[task] = {
    'model' : acc1,
    # 'bnn' : acc2
}

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
        [8.6732e-01, 9.0396e-02, 1.4075e-02, 1.4103e-02, 1.4101e-02],
        [9.3133e-01, 3.7106e-02, 1.0508e-02, 1.0526e-02, 1.0532e-02],
        [9.5613e-01, 1.8871e-02, 8.3210e-03, 8.3328e-03, 8.3425e-03],
        [6.5003e-01, 2.9476e-01, 1.8383e-02, 1.8428e-02, 1.8405e-02],
        [9.4625e-01, 2.5945e-02, 9.2546e-03, 9.2705e-03, 9.2783e-03],
        [8.5486e-01, 1.0132e-01, 1.4590e-02, 1.4619e-02, 1.4616e-02],
        [9.1720e-01, 4.8363e-02, 1.1464e-02, 1.1486e-02, 1.1491e-02],
        [7.6149e-01, 1.8719e-01, 1.7086e-02, 1.7124e-02, 1.7113e-02],
        [9.6252e-01, 1.4819e-02, 7.5409e-03, 7.5530e-03, 7.5624e-03],
        [9.2642e-01, 4.0794e-02, 1.0918e-02, 1.0934e-02, 1.0938e-02],
        [9.6793e-01, 1.1539e-02, 6.8323e-03, 6.8435e-03, 6.8532e-03],
        [9.9264e-01, 6.0153e-04, 2.2469e-03, 2.2493e-03, 2.2573e-03],
        [2.4415e-03, 9.9474e-01, 9.4079e-04, 9.4390e-04, 9.3843e-04],
        [7.2

In [91]:
# model = AudioClassifier(n_labels=n_labels).to(DEVICE)
# model.load_state_dict(torch.load('model_1.h5'))
# f1_score = F1Score(num_classes=n_labels, average='macro', multiclass=True)

# with torch.no_grad():
#     pred, targets = inference(model, train_dl)
#     score = f1_score(torch.from_numpy(pred).long(), torch.from_numpy(targets).long())
#     print(f'F1_score: {score}')


modelbnn = AudioClassifierBNN(n_labels=n_labels).to(DEVICE)
modelbnn.load_state_dict(torch.load('model_bnn_1.h5'))
f1_score = F1Score(num_classes=n_labels, average='macro', multiclass=True)

with torch.no_grad():
    pred, targets = inference(modelbnn, train_dl)
    score = f1_score(torch.from_numpy(pred).long(), torch.from_numpy(targets).long())
    print(f'F1_score: {score}')

#del train_dl, train_ds, val_dl, val_ds

Accuracy: 0.39, Loss: 1.13, Total items: 6080
F1_score: 0.18686869740486145


In [92]:
!rm bk/*

rm: cannot remove 'bk/*': No such file or directory


In [93]:
!cp -r bk* drive/MyDrive/birdclef/bk

In [94]:
loss_dict = {}
acc_dict = {}
task = '2'
print(f'Training Task {task}:')
print('Training size: ', len(data[task]['X_train']))
train_ds2 = SoundDS(data[task]['X_train'], data[task]['y_train'], mappings)
#val_ds = SoundDS(data[task]['X_val'], data[task]['y_val'], mappings, is_train=False)
train_dl2 = DataLoader(train_ds2, batch_size=32, shuffle=True)
#val_dl = DataLoader(val_ds, batch_size=32, shuffle=False)


# model = AudioClassifier(n_labels=n_labels).to(DEVICE)
# model.load_state_dict(torch.load('model_1.h5'))

# loss1, acc1 = train(model, train_dl, epochs=30)
# torch.save(model.state_dict(), f'model_{task}.h5')

# del model


modelbnn = AudioClassifierBNN(n_labels=n_labels).to(DEVICE)
modelbnn.load_state_dict(torch.load('model_bnn_1.h5'))
loss2, acc2= train(modelbnn, train_dl2, epochs=30, optim='meta', meta=20)
torch.save(modelbnn.state_dict(), f'model_bnn_{task}.h5')

del modelbnn

loss_dict[task] = { 
#    'model' : loss1,
    'bnn' : loss2
    }

acc_dict[task] = {
#    'model' : acc1,
    'bnn' : acc2
}

#del train_dl, train_ds, val_dl, val_ds

Training Task 2:
Training size:  298


KeyError: ignored

In [ ]:
# model = AudioClassifier(n_labels=n_labels).to(DEVICE)
# model.load_state_dict(torch.load('model_2.h5'))
f1_score = F1Score(num_classes=n_labels, average='macro', multiclass=True)

# with torch.no_grad():
#     pred, targets = inference(model, train_dl)
#     score = f1_score(torch.from_numpy(pred).long(), torch.from_numpy(targets).long())
#     print(f'F1_score: {score}')


modelbnn = AudioClassifierBNN(n_labels=n_labels).to(DEVICE)
modelbnn.load_state_dict(torch.load('model_bnn_2.h5'))

with torch.no_grad():
    pred, targets = inference(modelbnn, train_dl)
    score = f1_score(torch.from_numpy(pred).long(), torch.from_numpy(targets).long())
    print(f'F1_score: {score}')

#del train_dl, train_ds, val_dl, val_ds

In [ ]:
modelbnn.save_weight()

layers.cv1.weight :  Parameter containing:
tensor([[[[ 1., -1.,  1.,  1.,  1.],
          [ 1., -1., -1.,  1., -1.],
          [ 1.,  1., -1., -1., -1.],
          [-1., -1., -1., -1., -1.],
          [-1., -1., -1., -1.,  1.]]],


        [[[-1., -1., -1., -1., -1.],
          [-1., -1.,  1., -1., -1.],
          [-1., -1., -1., -1., -1.],
          [-1., -1., -1., -1., -1.],
          [-1., -1., -1., -1., -1.]]],


        [[[ 1., -1.,  1., -1., -1.],
          [-1., -1., -1., -1., -1.],
          [-1., -1., -1., -1., -1.],
          [-1., -1., -1., -1., -1.],
          [-1., -1., -1., -1.,  1.]]],


        ...,


        [[[-1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.],
          [ 1.,  1., -1.,  1.,  1.],
          [-1.,  1., -1.,  1.,  1.]]],


        [[[-1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.],
          [ 1.,  1., -1., -1., -1.],
          [ 1., -1., -1., -1., -1.],
          [-1.,  1., -1.,  1., -1.]]],

In [ ]:
model = AudioClassifier(n_labels=n_labels).to(DEVICE)
modelbnn = AudioClassifierBNN(n_labels=n_labels).to(DEVICE)
model.load_state_dict(torch.load('model_1.h5'))
modelbnn.load_state_dict(torch.load('model_bnn_1.h5'))
f1_score = F1Score(num_classes=n_labels, average='macro', multiclass=True)
print('Computing scores for both datasets...')
for task in data.keys():
    print(f'Task {task}:')
    test_ds = SoundDS(data[task]['X_test'], data[task]['y_test'], mappings, is_train=False)
    test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

    with torch.no_grad():
        print('Model: ')
        pred, targets = inference(model, test_dl)
        score = f1_score(torch.from_numpy(pred).long(), torch.from_numpy(targets).long())
        print(f'F1_score: {score}')

    with torch.no_grad():
        print('BNN: ')
        pred, targets = inference(modelbnn, test_dl)
        score = f1_score(torch.from_numpy(pred).long(), torch.from_numpy(targets).long())
        print(f'F1_score: {score}')

In [ ]:
import json
with open('loss.json', 'w') as fp:
  json.dump(loss_dict, fp, indent=4)
with open('acc.json', 'w') as fp:
  json.dump(acc_dict, fp, indent=4)